# Load Sagemaker, session and role object

In [1]:
import sagemaker

session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Upload data to S3

In [2]:
DATA_FILE = './data/iris.csv'
bucket_name = 'rsjainaimlmodels'
prefix = 'iris/data'
s3_output_path = 's3://'+bucket_name

print ('uploading data to s3')
s3_data_path = session.upload_data(path=DATA_FILE, 
                                   bucket=bucket_name, 
                                   key_prefix=prefix)
print ('data uploaded to -', s3_data_path)
print ('model would be uploaded to -', s3_output_path)

uploading data to s3
data uploaded to - s3://rsjainaimlmodels/iris/data/iris.csv
model would be uploaded to - s3://rsjainaimlmodels


# Create Sklearn Executor

In [14]:
from sagemaker.sklearn import SKLearn

executor = SKLearn(entry_point='train.py',
                   train_instance_type='ml.c4.xlarge',
                   role=role,
                   sagemaker_session=session,
                   output_path=s3_output_path
                  )

In [15]:
executor.fit(s3_data_path)

2020-05-09 11:29:43 Starting - Starting the training job...
2020-05-09 11:29:45 Starting - Launching requested ML instances......
2020-05-09 11:30:49 Starting - Preparing the instances for training...
2020-05-09 11:31:39 Downloading - Downloading input data...
2020-05-09 11:32:07 Training - Downloading the training image..
2020-05-09 11:32:39 Uploading - Uploading generated training model
2020-05-09 11:32:39 Completed - Training job completed
2020-05-09 11:32:27,803 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-09 11:32:27,805 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-09 11:32:27,815 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-09 11:32:28,238 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-05-09 11:32:28,239 sagemaker-containers INFO     Generating setup.cfg
2020-05-09 11:32:28,239 sagemaker-containers INFO

# Deploy the Model

In [16]:
predictor = executor.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
print('\nModel Deployed!')

---------------!
Model Deployed!


In [17]:
predictor.endpoint

'sagemaker-scikit-learn-2020-05-09-11-29-43-047'

# Predict

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv(DATA_FILE,engine='python')
X = data.iloc[:,1:5]
y = data.iloc[:,5]

train_x, test_x, train_y, test_y = train_test_split(X,y)
pred_y = predictor.predict(test_x.values)
print (pred_y)
print (test_y)

[2. 1. 0. 1. 1. 2. 1. 0. 0. 0. 2. 0. 1. 1. 0. 2. 1. 1. 2. 1. 2. 2. 1. 1.
 0. 2. 2. 2. 2. 2. 0. 1. 2. 2. 0. 0. 1. 1.]
149    2.0
78     1.0
45     0.0
60     1.0
89     1.0
136    2.0
92     1.0
16     0.0
41     0.0
43     0.0
126    2.0
25     0.0
82     1.0
98     1.0
24     0.0
105    2.0
61     1.0
133    2.0
142    2.0
53     1.0
139    2.0
114    2.0
65     1.0
56     1.0
22     0.0
143    2.0
121    2.0
112    2.0
146    2.0
84     1.0
39     0.0
85     1.0
100    2.0
111    2.0
2      0.0
47     0.0
62     1.0
91     1.0
Name: class, dtype: float64


# Delete Model Endpoint

In [21]:
session.delete_endpoint(predictor.endpoint)